In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import optuna

#加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
#transforms.ToTensor(): 将PIL图像或NumPy数组转换为PyTorch张量，并且会将图像的像素值从范围 [0, 255]缩放到 [0.0, 1.0]。transforms.Normalize((0.5,), (0.5,)): 将张量的每个通道的像素值进行标准化。像素值从 [0, 1] 归一化到 [-1, 1]。具体公式是：(x - mean) / std，其中mean和std是给定的均值和标准差。
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
#torchvision.datasets.MNIST 是一个用于加载MNIST数据集的类。

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)


#torch.utils.data.DataLoader 是PyTorch用于加载数据集的类。它能够处理批量数据并进行多线程加载。批次的数据量为100,训练随机打乱，测试不打乱。

In [16]:
# 定义神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()  #28*28二维图像展平成一维向量，size=784
        self.fc1 = nn.Linear(28 * 28, 30)  #784维映射到128维
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  #舍去50%的神经元
        self.fc2 = nn.Linear(30, 10)  #128维映射导10维

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def add_noise(inputs, noise_level):
    noise = torch.randn_like(inputs) * noise_level
    return inputs + noise

In [17]:
def objective(trial):
    noise_level = trial.suggest_float("noise_level", 0.0, 0.2)  # 优化噪声水平

    # 实例化模型、定义损失函数和优化器
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    num_epochs = 10  # 可以根据需要调整
    for epoch in range(num_epochs):
        model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = add_noise(inputs, noise_level)  # 添加噪声
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = add_noise(inputs, noise_level)  # 添加噪声
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [13]:
# 运行优化
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 获取最佳噪声水平
best_noise_level = study.best_params["noise_level"]
print("Best noise level:", best_noise_level)

In [14]:

# 实例化模型、定义损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 使用最佳噪声水平重新训练模型
model = Net().to(device)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = add_noise(inputs, best_noise_level)  # 使用最佳噪声水平添加噪声
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = add_noise(inputs, best_noise_level)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')